In [1]:
import pandas as pd
import numpy as np
import os
import json

pd.set_option('display.max_columns', 500)

In [2]:
'''
-read in the 2.Indices_Target.csv file
-filter on the NA-s in the snapshot_target column
-get the unique target nums
'''

filename='2.Indices_Target.csv'
df_target=pd.read_csv(f'C:/Users/511232/Desktop/Scorecard_coxcomb/{filename}', dtype='str')

#filter out the dataframe where snapshot_target is NA
df_na=df_target[df_target['snapshot_target'].isna()].copy()
target_num_NA=set(df_na['Target_number'])

In [3]:
'''
Code	Color
0	    Grey
1	    Red
2	    Yellow
3	    Green

-the scorecard.xlsx file, filter on color ('Color Scheme'=0) & ('Gap'=blank) & ('target' in target_num_NA)
-groupby ['geoareaname', 'target'] and apply a function that will take the average of 'Trend'
-and standardize the result of the average to be between [-10;+10]
formula=N= -10 + [(A+100)/10]
'''

filename='scorecard.xlsx'
#read in the scorecard.xlsx
df=pd.read_excel('C:/Users/511232/Desktop/Scorecard_coxcomb/scorecard.xlsx', dtype=str)
df['Trend'] = df['Trend'].astype(float)

#flag as 1 if (df['Color Scheme']=='0') & (df['Gap'].isna()) & (df['target'].isin(target_num_NA)) condition meets
cond=(df['Color Scheme']=='0') & (df['Gap'].isna()) & (df['target'].isin(target_num_NA))
df.loc[cond,'flag']=1

#filter color scheme=0 & Gap=blank
df_filtered=df[(df['Color Scheme']=='0') & (df['Gap'].isna()) & (df['target'].isin(target_num_NA))].copy()


def process(d):
    '''will calculate average trend and standardize the value'''
    trend_avg=d['Trend'].mean()
    trend_avg_standardized=-10+((trend_avg+100)/10)
    return(trend_avg_standardized)

#apply the function on the df_filtered
standardized_df=pd.DataFrame(df_filtered.groupby(['geoareaname','target'],group_keys=False).apply(process))
standardized_df.reset_index(drop=False, inplace=True)
standardized_df.columns=['geoareaname','target','standardized_value']
#add flag=1
standardized_df['flag']=1

#merge the standardized_df with df
result=pd.merge(df,standardized_df, how='left', on=['geoareaname','target', 'flag'])

In [9]:
'''
-generate coxcomb color using the following benchmarks:
if N is negative --> red, takes the value of 1
if N is postive but less or equal to 5.33 --> yellow, takes the value of 2
if N is greater than 5.33--> green, takes the value of 3"
'''
condition=[result['standardized_value']<0,(result['standardized_value']>0) & (result['standardized_value']<=5.33), result['standardized_value']>5.33]
colors=[1,2,3]
result['calculated_color']=np.select(condition,colors, default=np.nan)

result.to_excel('processed_file.xlsx',index=False)

## ---------------------------------------------------------------------------------

### discover the cutoff point for Yemen

In [2]:
os.chdir('C:/Users/511232/Desktop/Scorecard_coxcomb')
df=pd.read_excel('data Yemen.xlsx')

In [3]:
def report_on_series(d):
    # global idx
    # global ix
    
    #get the categories labels
    k=list(d.name)
    k=[str(i) for i in k]

    #report on the columns/names used in groupby
    c=groupby_vars
    categ_name=dict(zip(c,k))

    #update a dictionary that will track number of datapoints for each groupby dataframes
    series_count_key=f'{len(d)} datapoints'
    if series_count_key in series_count.keys():
        series_count[series_count_key]+=1
    else:
        series_count[series_count_key]=1

    if len(d)>=10:
                                    
            '''
            get the percentages of False and True from the boolean df['val'].pct_change()>0
            b will be like the below
            0     False
            1     False
            2     False
            3     False
            4      True
            '''
            #reset dataframe index
            d.reset_index(inplace=True)
            b=d['Value'].pct_change()>0
            freq=b.value_counts(normalize=True)*100

            '''check if True (or False) is between 45-65 %, that is if False and True are almost 50-50
            then there is a possibility of one chunk from the timeseries having monotonic percent change sign'''
            if True in freq.index:
                if (freq[True]>=40) & (freq[True]<=60):
                    '''get the index for ix of the True (or False)
                    ix will be like Int64Index([4, 8, 11, 12, 13, 14, 15, 16, 18, 19, 20], dtype='int64')'''
                    ix=b[b].index
                    cutoff_year_min=float(d["TimePeriod"][ix[0]])
                    cutoff_year_max=float(d["TimePeriod"][ix[-1]])
                    cutoff_years.append([cutoff_year_min,cutoff_year_max])
                 

            result_per_serie.append({'series':s, 'categories':categ_name, 'number of data_points':len(d), 'cutoff':cutoff_years})

In [4]:
#remove these from groupby vars
list_to_exclude=['BasePeriod', 'TimeCoverage', 'Time_Detail','LowerBound', 'UpperBound', 'FootNote','GeoInfoUrl', 'Source','Reporting Type', 'Sampling Stations']

#get unique SERIES
unique_series=list(df['SeriesCode'].unique())

result_per_serie=[] #initialize for each unique series
series_count={}     #it will count indicators count
cutoff_years=[]

for s in unique_series:
    #get the dataframe with SERIES == s
    df_s=df[df['SeriesCode']==s].copy()
    #get rid of null columns
    df_s.dropna(how='all',axis=1, inplace=True)
    #get the groupby vars which is the column names to groupby with
    cols=df_s.columns
    cols=set(cols).difference(set(list_to_exclude))
    groupby_vars=list(cols.difference({'TimePeriod', 'Value'}))
    # apply the function on the dataframe
    df_s.groupby(groupby_vars, as_index=True).apply(report_on_series)


#add the counter result to result_per_serie
result_per_serie.append(series_count)

#write it to a json file
with open('report.json', 'w') as outfile:
    json.dump(result_per_serie,outfile, indent=1)

In [ ]:
cutoff_years

In [13]:
from itertools import chain
flat_year_list=pd.Series(list(chain.from_iterable(cutoff_years)))

In [16]:
flat_year_list.value_counts().sort_values(ascending=False)

2001.0    58
2019.0    49
2018.0    35
2002.0    24
2021.0    20
2011.0    13
2015.0    10
2020.0    10
2003.0     8
2008.0     8
2006.0     7
2005.0     7
2017.0     6
2013.0     5
2004.0     4
2022.0     3
2010.0     2
2007.0     2
2009.0     2
2012.0     2
2016.0     1
dtype: int64